使用一对多的多分类策略，为每个状态建立单独的分类器，然后分别训练

未完成部分
- 配置全局参数可以任意指定要训练的模型和对应的数据集
- 配置全局参数可以针对指定的模型进行保存和读取
- 拆分训练集和验证集
- 配置整合函数main

In [ ]:
from StatusChecker.TraditionalStatusChecker import TraditionalStatusChecker
status_list = list(filter(lambda name: (name.startswith("ASC_STATUS_") and name != 'ASC_STATUS_UNKNOWN'),
                                  dir(TraditionalStatusChecker)))
status_list = list(map(lambda name: getattr(TraditionalStatusChecker, name), status_list))

print(status_list)
print(f"total {len(status_list)} status")
tag_dic = status_list
# tag_dic = ["level_selection",
#            "restore_sanity_medicine",
#            "restore_sanity_stone",
#            "team_up",
#            "fighting",
#            "battle_settlement",
#            "annihilation_settlement",
#            "level_up"]

current_train_tag = TraditionalStatusChecker.ASC_STATUS_FIGHTING

根据训练目标选取数据集
- 正向数据集选取目标标签，数量选取所有的或者指定阈值
- 反向数据集选取其余的标签数据，数量取正向数据集长度平均至每个TAG，不够的暂时全上

In [ ]:
import os
import torch

# 基础数据集路径
train_set_base_path = os.path.join(os.getcwd(), '..', 'dataset', 'processed')

# 所有可用标签
all_available_tag_data_path_list = os.listdir(train_set_base_path)

# 正向数据集路径
train_set_positive_data_path = os.path.join(train_set_base_path, current_train_tag)

# 反向数据集路径列表
train_set_negative_data_path_list = all_available_tag_data_path_list.copy()
train_set_negative_data_path_list.remove(current_train_tag)

#TODO 如果说数据量不平衡的话，最后那个数据会影响很多东西把


train_set_path = os.path.join(os.getcwd(), '..', 'dataset', 'processed')
train_file_list = os.listdir(train_set_path)

def map_func(filename:str) -> torch.Tensor:
    for index, tag in enumerate(tag_dic):
        if tag in filename:
            return torch.tensor(index, dtype=torch.long, device=torch.device('cuda'))


target_list = list(map(map_func, train_file_list))
print(len(train_file_list))
print(train_file_list[0])
print(tag_dic)
print(len(target_list))
print(target_list[2])

配置 DataSet

In [ ]:
import cv2 as cv
from torch.utils.data import Dataset

def default_loader(image_path:str) -> torch.Tensor:
    image_loaded = cv.imread(
        os.path.join(train_set_path, image_path),
        cv.IMREAD_GRAYSCALE)
    image_tensor = torch.tensor(image_loaded, device=torch.device('cuda'), dtype=torch.float32).unsqueeze(0)
    print(f"load image {image_path}")
    # print(image_tensor.size())
    return image_tensor

class TrainSet(Dataset):
    def __init__(self, image_name_list:list, targets:int, loader=default_loader)-> None:
        self.images = image_name_list
        self.targets = targets
        self.loader = loader
    def __getitem__(self, index: int):
        data = self.loader(self.images[index])
        target = self.targets[index]
        return data, target
    def __len__(self)-> int:
        return len(self.images)

def get_train_set() -> Dataset:

    train_set = TrainSet(train_file_list, target_list)
    print(train_set)
    print(train_set.__getitem__(3))
    print(train_set.__len__())
    return train_set

构建 DataLoader

In [ ]:
from torch.utils.data import DataLoader
def get_train_loader(train_set:Dataset) -> DataLoader:
    train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
    print(train_loader)
    print(iter(train_loader).next()[0].size())
    print(iter(train_loader).next()[1].size())
    return train_loader


定义模型

In [ ]:
import torch

class Lambda(torch.nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, xb):
        return self.func(xb)

def get_module():
    module = torch.nn.Sequential(
        torch.nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=3),
        torch.nn.ReLU(),
        torch.nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=3),
        torch.nn.ReLU(),
        torch.nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=3),
        torch.nn.ReLU(),
        torch.nn.AdaptiveAvgPool2d([16, 36]),
        Lambda(lambda xb: xb.view(-1, 10*16*36)),
        torch.nn.Linear(10*16*36, 16*36),
        torch.nn.Linear(16*36, 36),
        torch.nn.Linear(36, 1),
        torch.nn.Softmax(),
    )

    learn_rate = 0.1

    optimize = torch.optim.SGD(module.parameters(), lr=learn_rate)
    return module, optimize

def get_loss_function():
    loss_func = torch.nn.functional.binary_cross_entropy
    return loss_func


fit() 函数以及模型的保存和恢复

In [ ]:
import numpy

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def fit(epochs:int, module:torch.nn.Module,
        loss_function, optimize, train_loader, validate_loader):
    index = 0
    for epoch in range(epochs):
        for xb, yb in train_loader:
            index = index + 1
            if index%50 is 0:
                print(f"start index {index} data")
            loss_batch(module, loss_function, xb, yb, optimize)
        module.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(module, loss_function, xb, yb)
                  for xb, yb in validate_loader]
            )
        val_loss = numpy.sum(numpy.multiply(losses, nums)) / numpy.sum(nums)
        print(epoch, val_loss)


